In [22]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import warnings

In [23]:
!wget https://ocw.mit.edu/ans7870/6/6.006/s08/lecturenotes/files/t8.shakespeare.txt

--2016-10-17 21:48:37--  https://ocw.mit.edu/ans7870/6/6.006/s08/lecturenotes/files/t8.shakespeare.txt
Resolving ocw.mit.edu... 104.86.58.148, 2a02:26f0:e9:287::18a8, 2a02:26f0:e9:290::18a8
Connecting to ocw.mit.edu|104.86.58.148|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5458199 (5.2M) [text/plain]
Saving to: 't8.shakespeare.txt.1'

t8.shakespeare.txt. 100%[===================>]   5.21M  3.62MB/s    in 1.4s    

2016-10-17 21:48:40 (3.62 MB/s) - 't8.shakespeare.txt.1' saved [5458199/5458199]



In [24]:
with open('t8.shakespeare.txt') as file:
    raw_lines = [line.replace('\r',' ') for line in file.readlines()]
raw_text = ' '.join(raw_lines[100:])

define some parameters

In [25]:
text_size = len(raw_text)
vocab_size = len(set(raw_text))
num_neurons = 200
num_steps = 100
batch_size = 2000


function to generate batches of data

In [26]:
def gen_batch(raw_data, batch_size, num_steps):

    if batch_size % num_steps != 0:
        warnings.warn("Warning: batch size is not a multiple of num_steps.")
    
    raw_y = raw_data[1:] + raw_data[0]
    
    # partition raw data into batches and stack them vertically in a data matrix
    num_batch = data_length // batch_size
    data_x = np.zeros([num_batch, batch_size], dtype=np.int32)
    data_y = np.zeros([batch_size, batch_partition_length], dtype=np.int32)
    
    for i in range(num_batch):
        data_x[i] = raw_data[batch_size * i:batch_size * (i + 1)]
        data_y[i] = raw_y[batch_size * i:batch_size * (i + 1)]
    
    # further divide batch partitions into num_steps for truncated backprop
    batch_lines = batch_size // num_steps

    for i in range(num_batch):
        for n in range(batch_lines):
            x = data_x[i, n * num_steps:(n + 1) * num_steps]
            y = data_y[i, n * num_steps:(n + 1) * num_steps]
            yield (x, y)

### define the graph

In [37]:
x = tf.placeholder(tf.int32, [batch_size // num_steps, num_steps], name='input_placeholder')
y = tf.placeholder(tf.int32, [batch_size // num_steps, num_steps], name='labels_placeholder')
init_state = tf.zeros([batch_size // num_steps, num_neurons])

x_one_hot = tf.one_hot(x, vocab_size)
#rnn_inputs = tf.unpack(x_one_hot, axis=1)

cell = tf.nn.rnn_cell.BasicRNNCell(num_neurons)
rnn_outputs, final_state = tf.nn.dynamic_rnn(cell, x_one_hot, initial_state=init_state)

In [ ]:
max_length = int(y.get_shape()[1])
out_size = int(y.get_shape()[2])

weight = tf.Variable(tf.truncated_normal([num_neurons, out_size], stddev=0.1))
bias = tf.Variable(tf.constant(0.1, shape=[out_size]))

output = tf.reshape(output, [-1, num_neurons])
prediction = tf.nn.softmax(tf.matmul(output, weight) + bias)
prediction = tf.reshape(prediction, [-1, max_length, out_size])

cross_entropy = -tf.reduce_sum(
    target * tf.log(prediction), reduction_indices=[1])
cross_entropy = tf.reduce_mean(cross_entropy)

### train the network

In [38]:
def optimize(self):
        learning_rate = 0.003
        optimizer = tf.train.RMSPropOptimizer(learning_rate)
        return optimizer.minimize(self.cost)

In [ ]:
def train_network(num_epochs, num_steps, state_size=4, verbose=True):
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())
        training_losses = []
        for idx, epoch in enumerate(gen_epochs(num_epochs, num_steps)):
            training_loss = 0
            training_state = np.zeros((batch_size, state_size))
            if verbose:
                print("\nEPOCH", idx)
            for step, (X, Y) in enumerate(epoch):
                tr_losses, training_loss_, training_state, _ = \
                    sess.run([losses,
                              total_loss,
                              final_state,
                              train_step],
                                  feed_dict={x:X, y:Y, init_state:training_state})
                training_loss += training_loss_
                if step % 100 == 0 and step > 0:
                    if verbose:
                        print("Average loss at step", step,
                              "for last 250 steps:", training_loss/100)
                    training_losses.append(training_loss/100)
                    training_loss = 0

    return training_losses

In [ ]:

for i in range(100):
    batch = gen_batch(raw_data, batch_size, num_steps)
    sess.run(model.optimize, {data: batch, target: y, dropout: 0.5})
    error = sess.run(model.error, {data: test.data, target: test.target, dropout: 1})
    print('error {:3.1f}%'.format(100 * error))

In [ ]:
with tf.variable_scope('softmax'):
    W = tf.get_variable('W', [state_size, num_classes])
    b = tf.get_variable('b', [num_classes], initializer=tf.constant_initializer(0.0))
logits = [tf.matmul(rnn_outputs, W) + b ]
predictions = [tf.nn.softmax(logit) for logit in logits]

y_as_list = [tf.squeeze(i, squeeze_dims=[1]) for i in tf.split(1, num_steps, y)]

loss_weights = [tf.ones([batch_size]) for i in range(num_steps)]
losses = tf.nn.seq2seq.sequence_loss_by_example(logits, y_as_list, loss_weights)
total_loss = tf.reduce_mean(losses)
train_step = tf.train.AdagradOptimizer(learning_rate).minimize(total_loss)
